In [38]:
import os
import pandas as pd
import pyodbc

import numpy as np 
import matplotlib.pyplot as plt

from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns

Accessing SQL database for the ML model

In [148]:
server = 'gen10-data-fundamentals-22-02-sql-server.database.windows.net'
database = 'guinea-pigs-database'
username = 'gPigs'
password = 'GuineaPigs!'
driver='{SQL Server}'

cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
cursor.execute("SELECT * FROM NonNormalizedData WHERE [SBA Disaster Number] != '0'")
rows = cursor.fetchall()
c = [column[0] for column in cursor.description]
ma_df = pd.DataFrame([tuple(t) for t in rows], columns = c) 
ma_df

,Approved Amount Content,Approved Amount EIDL,Approved Amount Real Estate,County,Damaged Property Zip Code,Fines and Forfeits,Interest Revenue,Market Value,Property Tax,SBA Disaster Number,SBA EIDL Declaration Number,SBA Physical Declaration Number,State,Timestamp,Total Salaries & Wages,Unemployement,Verified Loss Content,Verified Loss Real Estate,Year,_c0
0,0.0,0.0,0.0,NASSAU,11771.0,351,125,1646.49,21006,NY-00130,13366.0,13365,NEW YORK,1652896912784,41174,5.9,5000.0,0.0,2013,2651
1,0.0,0.0,0.0,NASSAU,11804.0,351,125,1646.49,21006,NY-00130,13366.0,13365,NEW YORK,1652896912787,41174,5.9,7624.0,295854.0,2013,2651
2,0.0,0.0,0.0,NASSAU,11572.0,351,125,1646.49,21006,NY-00130,13366.0,13365,NEW YORK,1652896912789,41174,5.9,38325.0,165757.0,2013,2651
3,6297800.0,2190500.0,2341300.0,NASSAU,11572.0,351,125,1646.49,21006,NY-00130,13366.0,13365,NEW YORK,1652896912792,41174,5.9,14943432.0,9328552.0,2013,2651
4,0.0,0.0,0.0,NASSAU,11570.0,351,125,1646.49,21006,NY-00130,13366.0,13365,NEW YORK,1652896912795,41174,5.9,19600.0,10599.0,2013,2651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1903,0.0,0.0,0.0,COOK,60405.0,125,1888,935.9449999999999,11089,IL-00019,11470.0,11469,ILLINOIS,1652896883873,23083,10.5,4000.0,5676.0,2009,799
1904,800.0,0.0,23900.0,COOK,60104.0,125,1888,935.9449999999999,11089,IL-00019,11470.0,11469,ILLINOIS,1652896883917,23083,10.5,1950.0,47979.0,2009,799
1905,10500.0,2400.0,71500.0,COOK,60103.0,125,1888,935.9449999999999,11089,IL-00019,11470.0,11469,ILLINOIS,1652896883920,23083,10.5,32207.0,235939.0,2009,799
1906,173600.0,0.0,118900.0,KANE,60174.0,1049,6073,935.9449999999999,78029,IL-00019,11470.0,11469,ILLINOIS,1652896883925,43940,10.5,173607.0,118841.0,2009,853


In [149]:
from sklearn.preprocessing import RobustScaler
from sklearn import preprocessing
X = ma_df.drop(['Year','Unemployement', 'County', 'SBA Disaster Number', 'State', 'Timestamp','_c0','SBA EIDL Declaration Number','SBA Physical Declaration Number'], axis=1.)
scaler = preprocessing.RobustScaler()
robust_df = scaler.fit_transform(X)
robust_df = pd.DataFrame(robust_df)

Defining 'X' and 'y' variables. We are predicting Unemployment rate, so we are using that column as our 'y' value. Our 'X' value will be only the numerical data without the unemployment rate.


Executing the train test split method to split our dataset to be used for training and then testing the model. 

In [150]:
                                                                                                              

y = ma_df['Unemployement']
#X = ma_df.drop(['Unemployement', 'County', 'SBA Disaster Number', 'State', 'Timestamp','_c0','SBA EIDL Declaration Number','SBA Physical Declaration Number'], axis=1.)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.25, random_state=0)

In [151]:
model = ElasticNet(alpha=1.0, l1_ratio=0.5,max_iter=10000)
ma_enet_model = model.fit(X_train, y_train)

In [152]:
ma_enet_model.coef_ 

array([ 8.37828046e-07, -1.87645912e-06, -1.79561300e-07, -1.06669747e-05,
       -2.30220109e-05, -5.20601871e-05, -2.41323102e-03,  1.09823509e-06,
        2.36499577e-06, -2.35948158e-07,  1.01388882e-07])

In [153]:
ma_enet_model.intercept_

12.172627668306152

In [55]:
train_prediction = ma_enet_model.predict(X_train)[:10]

train_prediction


array([7.24150434, 8.85724372, 8.44064355, 8.20667814, 8.4626073 ,
       7.24855341, 8.19315676, 9.31139598, 8.16841175, 9.12301726])

In [56]:
model_prediction = ma_enet_model.predict(X_test)[:10]

model_prediction

array([9.22924982, 8.17911162, 9.32294388, 8.47987663, 8.99168539,
       8.94632666, 8.48856844, 8.97483323, 8.33136208, 8.65058145])

In [57]:
from sklearn.model_selection import GridSearchCV
import numpy as np

In [10]:

clf = GridSearchCV(model, {'alpha': [0.5,1,5,10],'l1_ratio': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]},cv=10)
clf.fit(X_train, y_train)
clf.cv_results_



{'mean_fit_time': array([0.00870976, 0.00763164, 0.00782959, 0.00730331, 0.00710301,
        0.00787315, 0.00775969, 0.00751786, 0.00802233, 0.00838935,
        0.00736241, 0.00737772, 0.00721242, 0.0073802 , 0.00920603,
        0.00906773, 0.00872197, 0.00870771, 0.00993657, 0.00940249,
        0.00874102, 0.00844615, 0.00752685, 0.00755587, 0.00707107,
        0.00756216, 0.00737367, 0.00733495, 0.00811522, 0.00780795,
        0.00734076, 0.009303  , 0.00806415, 0.00754037, 0.00728686,
        0.00674965]),
 'std_fit_time': array([0.00173775, 0.00120526, 0.00160554, 0.00141931, 0.00092786,
        0.0016995 , 0.0009367 , 0.00109465, 0.00115968, 0.00122566,
        0.00158653, 0.00112679, 0.00120338, 0.00105109, 0.00140823,
        0.00118625, 0.00119075, 0.00103235, 0.00140895, 0.00156681,
        0.00105268, 0.00162046, 0.00145591, 0.0013729 , 0.00148584,
        0.00107454, 0.00123673, 0.00174995, 0.00105622, 0.00127583,
        0.00090313, 0.0013091 , 0.00182463, 0.0010596 , 0.001

In [11]:
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.008710,0.001738,0.001903,0.000443,0.5,0.1,"{'alpha': 0.5, 'l1_ratio': 0.1}",0.245261,0.215050,0.176462,0.124090,0.244363,0.169683,0.178458,0.170101,0.110434,0.277213,0.191111,0.051020,1
1,0.007632,0.001205,0.002073,0.000301,0.5,0.2,"{'alpha': 0.5, 'l1_ratio': 0.2}",0.243145,0.212866,0.172006,0.119310,0.240145,0.164743,0.174425,0.164535,0.107433,0.274473,0.187308,0.051567,2
2,0.007830,0.001606,0.001643,0.000452,0.5,0.3,"{'alpha': 0.5, 'l1_ratio': 0.3}",0.242647,0.212370,0.171766,0.119277,0.239802,0.164517,0.174054,0.164471,0.107055,0.273980,0.186994,0.051463,27
3,0.007303,0.001419,0.001919,0.000508,0.5,0.4,"{'alpha': 0.5, 'l1_ratio': 0.4}",0.242645,0.212337,0.171777,0.119327,0.239791,0.164552,0.174042,0.164516,0.107067,0.273914,0.186997,0.051437,25
4,0.007103,0.000928,0.001327,0.000490,0.5,0.5,"{'alpha': 0.5, 'l1_ratio': 0.5}",0.242644,0.212304,0.171787,0.119378,0.239779,0.164587,0.174030,0.164560,0.107079,0.273848,0.187000,0.051411,22
5,0.007873,0.001700,0.002065,0.000183,0.5,0.6,"{'alpha': 0.5, 'l1_ratio': 0.6}",0.242642,0.212271,0.171798,0.119428,0.239768,0.164622,0.174018,0.164604,0.107091,0.273782,0.187002,0.051384,21
6,0.007760,0.000937,0.001869,0.000377,0.5,0.7,"{'alpha': 0.5, 'l1_ratio': 0.7}",0.242640,0.212237,0.171808,0.119478,0.239756,0.164657,0.174006,0.164648,0.107103,0.273716,0.187005,0.051358,19
7,0.007518,0.001095,0.001929,0.000501,0.5,0.8,"{'alpha': 0.5, 'l1_ratio': 0.8}",0.242638,0.212203,0.171819,0.119528,0.239744,0.164692,0.173993,0.164692,0.107114,0.273649,0.187007,0.051332,18
8,0.008022,0.001160,0.001724,0.000500,0.5,0.9,"{'alpha': 0.5, 'l1_ratio': 0.9}",0.242636,0.212170,0.171829,0.119578,0.239732,0.164727,0.173981,0.164736,0.107126,0.273583,0.187010,0.051306,16
9,0.008389,0.001226,0.001919,0.000246,1,0.1,"{'alpha': 1, 'l1_ratio': 0.1}",0.242889,0.212626,0.171876,0.119269,0.239974,0.164609,0.174239,0.164481,0.107232,0.274249,0.187145,0.051526,3


In [143]:
alphas = np.logspace(0,1,100)

In [144]:
alphas

array([ 1.        ,  1.02353102,  1.04761575,  1.07226722,  1.09749877,
        1.12332403,  1.149757  ,  1.17681195,  1.20450354,  1.23284674,
        1.26185688,  1.29154967,  1.32194115,  1.35304777,  1.38488637,
        1.41747416,  1.45082878,  1.48496826,  1.51991108,  1.55567614,
        1.59228279,  1.62975083,  1.66810054,  1.70735265,  1.7475284 ,
        1.78864953,  1.83073828,  1.87381742,  1.91791026,  1.96304065,
        2.009233  ,  2.05651231,  2.10490414,  2.15443469,  2.20513074,
        2.25701972,  2.3101297 ,  2.36448941,  2.42012826,  2.47707636,
        2.53536449,  2.59502421,  2.65608778,  2.71858824,  2.7825594 ,
        2.84803587,  2.91505306,  2.98364724,  3.05385551,  3.12571585,
        3.19926714,  3.27454916,  3.35160265,  3.43046929,  3.51119173,
        3.59381366,  3.67837977,  3.76493581,  3.85352859,  3.94420606,
        4.03701726,  4.1320124 ,  4.22924287,  4.32876128,  4.43062146,
        4.53487851,  4.64158883,  4.75081016,  4.86260158,  4.97

In [154]:
clf2 = GridSearchCV(model, {'alpha': alphas,'l1_ratio': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],'normalize':[True,False],},cv=10)
clf2.fit(X_train, y_train)
clf2.cv_results_
df2 = pd.DataFrame(clf2.cv_results_)
df2

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,param_normalize,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.006191,0.000965,0.002113,0.000548,1.0,0.1,True,"{'alpha': 1.0, 'l1_ratio': 0.1, 'normalize': T...",-0.000258,-0.001987,...,-0.032379,-0.001310,-0.000002,-0.001821,-0.024600,-0.004901,-0.000140,-0.006875,0.011026,901
1,0.006856,0.000751,0.001700,0.000458,1.0,0.1,False,"{'alpha': 1.0, 'l1_ratio': 0.1, 'normalize': F...",0.016929,0.157598,...,-0.019751,-0.003835,0.048441,0.020554,0.071009,0.101265,0.076119,0.056244,0.051586,900
2,0.005560,0.000506,0.001632,0.000521,1.0,0.2,True,"{'alpha': 1.0, 'l1_ratio': 0.2, 'normalize': T...",-0.000258,-0.001987,...,-0.032379,-0.001310,-0.000002,-0.001821,-0.024600,-0.004901,-0.000140,-0.006875,0.011026,901
3,0.006601,0.000813,0.001691,0.000426,1.0,0.2,False,"{'alpha': 1.0, 'l1_ratio': 0.2, 'normalize': F...",0.016970,0.157506,...,-0.019718,-0.003754,0.048462,0.020585,0.070984,0.101238,0.076098,0.056248,0.051544,870
4,0.005713,0.000714,0.001563,0.000766,1.0,0.3,True,"{'alpha': 1.0, 'l1_ratio': 0.3, 'normalize': T...",-0.000258,-0.001987,...,-0.032379,-0.001310,-0.000002,-0.001821,-0.024600,-0.004901,-0.000140,-0.006875,0.011026,901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,0.006368,0.000776,0.001623,0.000433,10.0,0.7,False,"{'alpha': 10.0, 'l1_ratio': 0.7, 'normalize': ...",0.019576,0.151038,...,-0.017592,0.001550,0.049728,0.022522,0.069128,0.099274,0.074484,0.056323,0.048676,160
1796,0.006123,0.000760,0.001851,0.000692,10.0,0.8,True,"{'alpha': 10.0, 'l1_ratio': 0.8, 'normalize': ...",-0.000258,-0.001987,...,-0.032379,-0.001310,-0.000002,-0.001821,-0.024600,-0.004901,-0.000140,-0.006875,0.011026,901
1797,0.006404,0.000760,0.001484,0.000493,10.0,0.8,False,"{'alpha': 10.0, 'l1_ratio': 0.8, 'normalize': ...",0.019929,0.150055,...,-0.017305,0.002287,0.049887,0.022781,0.068830,0.098961,0.074215,0.056306,0.048258,281
1798,0.005432,0.000782,0.001741,0.000430,10.0,0.9,True,"{'alpha': 10.0, 'l1_ratio': 0.9, 'normalize': ...",-0.000258,-0.001987,...,-0.032379,-0.001310,-0.000002,-0.001821,-0.024600,-0.004901,-0.000140,-0.006875,0.011026,901


In [155]:
df2.sort_values(by=['rank_test_score'])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,param_normalize,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
1791,0.006004,0.000624,0.001932,0.000509,10.0,0.5,False,"{'alpha': 10.0, 'l1_ratio': 0.5, 'normalize': ...",0.018848,0.152977,...,-0.018186,0.000044,0.049389,0.021985,0.069703,0.099881,0.074996,0.056337,0.049515,1
1667,0.006392,0.000824,0.001722,0.000612,8.497534,0.6,False,"{'alpha': 8.497534359086444, 'l1_ratio': 0.6, ...",0.018883,0.152886,...,-0.018157,0.000117,0.049406,0.022011,0.069677,0.099853,0.074972,0.056337,0.049475,2
1543,0.006788,0.001148,0.001592,0.000557,7.220809,0.7,False,"{'alpha': 7.220809018385466, 'l1_ratio': 0.7, ...",0.018866,0.152931,...,-0.018171,0.000081,0.049397,0.021998,0.069690,0.099867,0.074984,0.056337,0.049494,3
1561,0.006909,0.001223,0.001735,0.000360,7.390722,0.7,False,"{'alpha': 7.390722033525779, 'l1_ratio': 0.7, ...",0.018910,0.152816,...,-0.018135,0.000172,0.049418,0.022031,0.069656,0.099831,0.074954,0.056337,0.049444,4
1649,0.007550,0.000905,0.001789,0.000391,8.302176,0.6,False,"{'alpha': 8.302175681319746, 'l1_ratio': 0.6, ...",0.018840,0.152999,...,-0.018193,0.000027,0.049385,0.021979,0.069710,0.099888,0.075001,0.056337,0.049524,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034,0.005179,0.000474,0.001541,0.000474,3.764936,0.5,True,"{'alpha': 3.7649358067924683, 'l1_ratio': 0.5,...",-0.000258,-0.001987,...,-0.032379,-0.001310,-0.000002,-0.001821,-0.024600,-0.004901,-0.000140,-0.006875,0.011026,901
324,0.006209,0.000570,0.001532,0.000464,1.519911,0.1,True,"{'alpha': 1.5199110829529336, 'l1_ratio': 0.1,...",-0.000258,-0.001987,...,-0.032379,-0.001310,-0.000002,-0.001821,-0.024600,-0.004901,-0.000140,-0.006875,0.011026,901
1036,0.005682,0.000992,0.001745,0.000439,3.764936,0.6,True,"{'alpha': 3.7649358067924683, 'l1_ratio': 0.6,...",-0.000258,-0.001987,...,-0.032379,-0.001310,-0.000002,-0.001821,-0.024600,-0.004901,-0.000140,-0.006875,0.011026,901
1038,0.005616,0.001123,0.001711,0.000473,3.764936,0.7,True,"{'alpha': 3.7649358067924683, 'l1_ratio': 0.7,...",-0.000258,-0.001987,...,-0.032379,-0.001310,-0.000002,-0.001821,-0.024600,-0.004901,-0.000140,-0.006875,0.011026,901


In [127]:
alphas = np.logspace(-1,1.2,100)
alphas

array([ 0.1       ,  0.10525003,  0.11077569,  0.11659144,  0.12271252,
        0.12915497,  0.13593564,  0.1430723 ,  0.15058364,  0.15848932,
        0.16681005,  0.17556763,  0.18478498,  0.19448624,  0.20469683,
        0.21544347,  0.22675431,  0.23865898,  0.25118864,  0.26437612,
        0.27825594,  0.29286446,  0.30823992,  0.32442261,  0.34145489,
        0.35938137,  0.37824899,  0.39810717,  0.41900791,  0.44100595,
        0.46415888,  0.48852736,  0.51417518,  0.54116953,  0.56958108,
        0.59948425,  0.63095734,  0.66408279,  0.69894732,  0.73564225,
        0.77426368,  0.81491275,  0.8576959 ,  0.90272518,  0.95011851,
        1.        ,  1.05250029,  1.10775685,  1.1659144 ,  1.22712524,
        1.29154967,  1.35935639,  1.43072299,  1.50583635,  1.58489319,
        1.66810054,  1.75567629,  1.8478498 ,  1.94486244,  2.04696827,
        2.15443469,  2.26754313,  2.38658979,  2.51188643,  2.64376119,
        2.7825594 ,  2.92864456,  3.08239924,  3.24422608,  3.41

In [129]:
clf3 = GridSearchCV(model, {'alpha': alphas,'l1_ratio': [0.09,0.091,0.092,0.093,0.094,0.095,0.096,0.097,0.098,0.099,0.11,0.12,0.13,0.14],},cv=10)
clf3.fit(X_train, y_train)
clf3.cv_results_
df3 = pd.DataFrame(clf3.cv_results_)
df3.sort_values(by=['rank_test_score'])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
844,0.005783,0.000626,0.001645,0.000483,11.659144,0.8,"{'alpha': 11.659144011798324, 'l1_ratio': 0.8}",0.074241,0.052740,0.017934,0.014155,0.092543,0.013445,0.047831,0.066107,0.096029,0.060597,0.053562,0.029022,1
827,0.005949,0.000405,0.001435,0.000546,10.525003,0.9,"{'alpha': 10.525002852777327, 'l1_ratio': 0.9}",0.074248,0.052746,0.017992,0.014125,0.092533,0.013499,0.047820,0.066112,0.095978,0.060570,0.053562,0.029003,2
896,0.005977,0.000468,0.001407,0.000484,15.848932,0.6,"{'alpha': 15.848931924611133, 'l1_ratio': 0.6}",0.074249,0.052747,0.018012,0.014114,0.092529,0.013518,0.047817,0.066114,0.095960,0.060560,0.053562,0.028996,3
870,0.005871,0.000460,0.001632,0.000494,13.593564,0.7,"{'alpha': 13.593563908785255, 'l1_ratio': 0.7}",0.074249,0.052748,0.018013,0.014114,0.092529,0.013518,0.047817,0.066114,0.095960,0.060560,0.053562,0.028996,4
887,0.006248,0.000754,0.001747,0.000392,15.058364,0.6,"{'alpha': 15.058363542798414, 'l1_ratio': 0.6}",0.074228,0.052729,0.017818,0.014214,0.092564,0.013339,0.047852,0.066096,0.096131,0.060652,0.053562,0.029060,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,0.006268,0.000931,0.001449,0.000442,0.122713,0.1,"{'alpha': 0.122712523985119, 'l1_ratio': 0.1}",0.073576,0.051939,0.013769,0.015876,0.092900,0.009590,0.048138,0.065347,0.099058,0.062102,0.053230,0.030328,896
27,0.006169,0.000777,0.001846,0.000484,0.116591,0.1,"{'alpha': 0.11659144011798317, 'l1_ratio': 0.1}",0.073576,0.051939,0.013769,0.015876,0.092900,0.009589,0.048138,0.065347,0.099058,0.062102,0.053229,0.030328,897
18,0.006720,0.000635,0.001595,0.000587,0.110776,0.1,"{'alpha': 0.11077568505097089, 'l1_ratio': 0.1}",0.073576,0.051939,0.013769,0.015876,0.092900,0.009589,0.048138,0.065347,0.099058,0.062102,0.053229,0.030328,898
9,0.006973,0.001000,0.001638,0.000494,0.10525,0.1,"{'alpha': 0.10525002852777329, 'l1_ratio': 0.1}",0.073576,0.051939,0.013769,0.015876,0.092900,0.009589,0.048138,0.065347,0.099058,0.062102,0.053229,0.030328,899


In [92]:
alphas = np.logspace(-1.5,1.1,100)
alphas

array([ 0.1       ,  0.10500552,  0.11026159,  0.11578075,  0.12157618,
        0.12766169,  0.13405182,  0.14076181,  0.14780767,  0.15520621,
        0.16297508,  0.17113283,  0.17969891,  0.18869378,  0.19813888,
        0.20805675,  0.21847107,  0.22940668,  0.24088967,  0.25294745,
        0.26560878,  0.27890387,  0.29286446,  0.30752384,  0.322917  ,
        0.33908067,  0.35605341,  0.37387573,  0.39259015,  0.41224132,
        0.43287613,  0.45454382,  0.47729609,  0.50118723,  0.52627425,
        0.552617  ,  0.58027834,  0.60932428,  0.63982412,  0.67185063,
        0.70548023,  0.74079317,  0.7778737 ,  0.81681031,  0.8576959 ,
        0.90062802,  0.94570912,  0.99304676,  1.04275389,  1.09494912,
        1.149757  ,  1.20730829,  1.26774032,  1.33119729,  1.39783061,
        1.46779927,  1.54127022,  1.61841878,  1.69942902,  1.78449424,
        1.87381742,  1.96761169,  2.06610084,  2.16951989,  2.2781156 ,
        2.39214708,  2.51188643,  2.63761936,  2.76964586,  2.90

In [110]:
ratio = np.logspace(-2,0.0001,100)
ratio

array([0.01      , 0.01047618, 0.01097504, 0.01149765, 0.01204515,
       0.01261872, 0.0132196 , 0.01384909, 0.01450856, 0.01519943,
       0.0159232 , 0.01668143, 0.01747577, 0.01830794, 0.01917973,
       0.02009303, 0.02104982, 0.02205218, 0.02310226, 0.02420235,
       0.02535482, 0.02656218, 0.02782702, 0.02915209, 0.03054026,
       0.03199453, 0.03351805, 0.03511412, 0.03678619, 0.03853789,
       0.04037299, 0.04229548, 0.04430951, 0.04641945, 0.04862986,
       0.05094553, 0.05337146, 0.05591291, 0.05857538, 0.06136464,
       0.06428671, 0.06734793, 0.07055491, 0.07391461, 0.07743429,
       0.08112157, 0.08498444, 0.08903124, 0.09327075, 0.09771213,
       0.10236501, 0.10723944, 0.11234599, 0.1176957 , 0.12330016,
       0.12917149, 0.1353224 , 0.14176621, 0.14851686, 0.15558896,
       0.16299783, 0.17075949, 0.17889075, 0.1874092 , 0.19633329,
       0.20568232, 0.21547654, 0.22573715, 0.23648634, 0.24774739,
       0.25954467, 0.27190372, 0.28485128, 0.29841539, 0.31262

In [112]:
clf4 = GridSearchCV(model, {'alpha': alphas,'l1_ratio': [0.01, 0.01047618, 0.01097504, 0.01149765, 0.01204515,
       0.01261872, 0.0132196 , 0.01384909, 0.01450856, 0.01519943,
       0.0159232 , 0.01668143, 0.01747577, 0.01830794, 0.01917973,
       0.02009303, 0.02104982, 0.02205218, 0.02310226, 0.02420235,
       0.02535482, 0.02656218, 0.02782702, 0.02915209, 0.03054026,
       0.03199453, 0.03351805, 0.03511412, 0.03678619, 0.03853789,
       0.04037299, 0.04229548, 0.04430951, 0.04641945, 0.04862986,
       0.05094553, 0.05337146, 0.05591291, 0.05857538, 0.06136464,
       0.06428671, 0.06734793, 0.07055491, 0.07391461, 0.07743429,
       0.08112157, 0.08498444, 0.08903124, 0.09327075, 0.09771213,
       0.10236501, 0.10723944, 0.11234599, 0.1176957 , 0.12330016,
       0.12917149, 0.1353224 , 0.14176621, 0.14851686, 0.15558896,
       0.16299783, 0.17075949, 0.17889075, 0.1874092 , 0.19633329,
       0.20568232, 0.21547654, 0.22573715, 0.23648634, 0.24774739,
       0.25954467, 0.27190372, 0.28485128, 0.29841539, 0.31262539,
       0.32751204, 0.34310757, 0.35944573, 0.37656189, 0.39449308,
       0.41327813, 0.43295769, 0.45357435, 0.47517274, 0.4977996 ,
       0.52150392, 0.54633699, 0.57235257, 0.59960696, 0.62815916,
       0.65807096, 0.68940711, 0.72223543, 0.75662697, 0.79265618,
       0.83040103, 0.86994322, 0.91136834, 0.95476605]},cv=10)
clf4.fit(X_train, y_train)
clf4.cv_results_
df4 = pd.DataFrame(clf4.cv_results_)
df4.sort_values(by=['rank_test_score'])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
98,0.006475,0.000469,0.001517,0.000535,0.1,0.954766,"{'alpha': 0.1, 'l1_ratio': 0.95476605}",0.261635,0.203846,0.178624,0.158251,0.285095,0.209604,0.224787,0.201942,0.135860,0.177864,0.203751,0.042928,1
97,0.006508,0.000806,0.001615,0.000437,0.1,0.911368,"{'alpha': 0.1, 'l1_ratio': 0.91136834}",0.261623,0.203834,0.178616,0.158239,0.285089,0.209588,0.224780,0.201928,0.135883,0.177865,0.203745,0.042923,2
96,0.006476,0.000442,0.001533,0.000549,0.1,0.869943,"{'alpha': 0.1, 'l1_ratio': 0.86994322}",0.261612,0.203822,0.178609,0.158229,0.285084,0.209572,0.224774,0.201915,0.135904,0.177865,0.203738,0.042918,3
95,0.006415,0.000655,0.001804,0.000527,0.1,0.830401,"{'alpha': 0.1, 'l1_ratio': 0.83040103}",0.261601,0.203812,0.178601,0.158219,0.285079,0.209557,0.224768,0.201902,0.135923,0.177865,0.203733,0.042914,4
94,0.006396,0.000532,0.001662,0.000429,0.1,0.792656,"{'alpha': 0.1, 'l1_ratio': 0.79265618}",0.261591,0.203801,0.178595,0.158210,0.285075,0.209542,0.224762,0.201890,0.135942,0.177866,0.203727,0.042910,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6018,0.006169,0.000401,0.001561,0.000477,1.873817,0.376562,"{'alpha': 1.8738174228603839, 'l1_ratio': 0.37...",0.073643,0.052029,0.014105,0.015764,0.092896,0.009902,0.048141,0.065432,0.098853,0.062010,0.053277,0.030227,9896
4157,0.006328,0.000469,0.001443,0.000448,0.740793,0.954766,"{'alpha': 0.7407931702277049, 'l1_ratio': 0.95...",0.073643,0.052029,0.014104,0.015764,0.092896,0.009901,0.048141,0.065432,0.098854,0.062010,0.053277,0.030227,9897
5920,0.006210,0.000396,0.001540,0.000468,1.784494,0.394493,"{'alpha': 1.7844942442702214, 'l1_ratio': 0.39...",0.073643,0.052029,0.014104,0.015764,0.092896,0.009901,0.048141,0.065431,0.098854,0.062010,0.053277,0.030227,9898
5822,0.005840,0.000542,0.001366,0.000386,1.699429,0.413278,"{'alpha': 1.6994290206633513, 'l1_ratio': 0.41...",0.073642,0.052029,0.014103,0.015764,0.092896,0.009900,0.048141,0.065431,0.098854,0.062010,0.053277,0.030227,9899


In [94]:
final_model = ElasticNet(alpha=2.888059, l1_ratio=0.93,max_iter=10000)

In [18]:
final_model.fit(X_train, y_train)

ElasticNet(alpha=2.888059, l1_ratio=0.93, max_iter=10000)

In [19]:
final_model.coef_

array([ 6.16378145e-07, -1.61976824e-06, -1.63205926e-07, -2.41025710e-05,
        7.36932459e-06, -1.32090349e-05, -2.89889074e-03, -2.79725448e-07,
        1.08532395e-04,  4.22760063e-08,  3.88752629e-07, -2.71394658e-07,
        1.50979104e-07,  0.00000000e+00, -3.73080671e-04])

In [20]:
final_model.intercept_

12.652972218797021

In [21]:
finalmodelpredictions = final_model.predict(X_test)
finalmodelpredictions

array([6.73016608, 8.33480704, 6.59051097, ..., 6.56402109, 9.06747554,
       9.66940218])

In [29]:
len(finalmodelpredictions)

1366

In [22]:
(y_test)[:10]

678      9.7
3865     7.9
5005     5.3
4422     8.2
1852     9.2
2797     2.8
863     11.6
4302     9.1
1859     9.2
2407    15.3
Name: Unemployement, dtype: object

In [23]:
from sklearn.metrics import accuracy_score

In [24]:
from sklearn.metrics import mean_squared_error

In [25]:
np.sqrt(mean_squared_error(y_test,finalmodelpredictions))

2.515851563861444

In [23]:
from sklearn.metrics import r2_score

In [27]:
r2_score(y_test,finalmodelpredictions)

0.18424102443808632

In [147]:
f2= ElasticNet(alpha=1, l1_ratio=0.2,max_iter=10000)
f2.fit(X_train, y_train)
f2predictions = f2.predict(X_test)
r2_score(y_test,f2predictions)

-0.2774446747650956

In [132]:
f3= ElasticNet(alpha=11.659144, l1_ratio=0.8,max_iter=10000)
f3.fit(X_train, y_train)
f3predictions = f3.predict(X_test)
r2_score(y_test,f3predictions)

0.07266247375317281

In [113]:
f4= ElasticNet(alpha=0.1, l1_ratio=0.954766,max_iter=10000)
f4.fit(X_train, y_train)
f4predictions = f4.predict(X_test)
r2_score(y_test,f4predictions)

0.1989894640897265

In [64]:
thefinalmodel = f4